# Implementing different Callbacks on IMDB Dataset

Here are some examples of ways you can use callbacks:

 Model checkpointing—Saving the current weights of the model at different points
during training.

 Early stopping—Interrupting training when the validation loss is no longer
improving (and of course, saving the best model obtained during training).

 Dynamically adjusting the value of certain parameters during training—Such as the
learning rate of the optimizer.

 Logging training and validation metrics during training, or visualizing the representations learned by the model as they’re updated—The Keras progress bar that you’re
familiar with is a callback!

The keras.callbacks module includes a number of built-in callbacks (this is not an
exhaustive list):

keras.callbacks.ModelCheckpoint

keras.callbacks.EarlyStopping

keras.callbacks.LearningRateScheduler

keras.callbacks.ReduceLROnPlateau

keras.callbacks.CSVLogger

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras import layers

max_features = 2000
max_len = 500

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)

model = tf.keras.models.Sequential()
model.add(layers.Embedding(max_features, 128,input_length=max_len,name='embed'))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))
#model.summary()

model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])

17465344/17464789 [==============================] - 0s 0us/step


# Early Stopping and model Checkpoint
We can pass more than one callbacks in a list

In [ ]:
callbacks_list = [tf.keras.callbacks.EarlyStopping(   # stop if not improving
                monitor='acc',patience=1,),           # monitor validation accuracy
                tf.keras.callbacks.ModelCheckpoint(
                filepath='my_model.h5',
                monitor='val_loss',                   # only save best weights
                save_best_only=True,)]                # when vall loss is improved

In [ ]:
history = model.fit(x_train, y_train,
                    epochs=2,
                    batch_size=128,
                    validation_split=0.2,
                    callbacks=callbacks_list)

Epoch 1/2
157/157 [==============================] - 75s 480ms/step - loss: 0.6174 - acc: 0.6837 - val_loss: 0.4078 - val_acc: 0.8308
Epoch 2/2
157/157 [==============================] - 76s 484ms/step - loss: 0.4369 - acc: 0.8480 - val_loss: 0.4545 - val_acc: 0.8592


# THE REDUCELRONPLATEAU CALLBACK
Use this callback to reduce the learning rate when the validation loss has
stopped improving. Reducing or increasing the learning rate in case of a loss plateau to get out of local minima during training.
###########################

Platue straight surface hote hai aur gradiant us se bhr nahe araha hota
qk gradiant ko neche jana hota hai aur woh surface sedhe hote hai

In [ ]:
callbacks_list = [tf.keras.callbacks.ReduceLROnPlateau(
                monitor='val_loss',factor=0.1,         # lr ko .1 se multiply kerdo (kam kerdo)
                patience=10,)]                         # 10 epochs tak improvement nahe hoto stop hojai

In [ ]:
history = model.fit(x_train, y_train,
                    epochs=2,
                    batch_size=128,
                    validation_split=0.2,
                    callbacks=callbacks_list)

Epoch 1/2
157/157 [==============================] - 75s 481ms/step - loss: 0.3939 - acc: 0.8766 - val_loss: 0.4354 - val_acc: 0.8606
Epoch 2/2
157/157 [==============================] - 76s 483ms/step - loss: 0.3435 - acc: 0.9022 - val_loss: 0.6631 - val_acc: 0.8434


# We can create our own model

In [ ]:
from tensorflow import keras
import numpy as np
class ActivationLogger(keras.callbacks.Callback):
            
    def on_epoch_end(self, epoch, logs=None):
        print("<<<<<< Epoch end >>>>>>>")
        
        
    def on_epoch_begin(self, epoch, logs=None):
        print("<<<<<< Epoch begin >>>>>>>")
        
        
    def on_train_begin(self, epoch, logs=None):
        print("<<<<<<< Training begin >>>>")

        
    def on_train_end(self, epoch, logs=None):
        print("<<<<<<< Training ended >>>>")

callbacks_list = [ActivationLogger()]

In [ ]:
history = model.fit(x_train, y_train,
                    epochs=2,
                    batch_size=128,
                    validation_split=0.2,
                    callbacks=callbacks_list)

<<<<<<< Training begin >>>>
<<<<<< Epoch begin >>>>>>>
Epoch 1/2
157/157 [==============================] - 76s 484ms/step - loss: 0.2128 - acc: 0.9542 - val_loss: 0.7353 - val_acc: 0.8620
<<<<<< Epoch begin >>>>>>>
Epoch 2/2
157/157 [==============================] - 77s 489ms/step - loss: 0.1788 - acc: 0.9698 - val_loss: 0.9845 - val_acc: 0.8460
<<<<<<< Training ended >>>>



# LearningRateScheduler

seem same as REDUCELRONPLATEAU to me

https://stackoverflow.com/questions/39779710/setting-up-a-learningratescheduler-in-keras

In [ ]:
def lr_scheduler(epoch, lr):
    decay_rate = 0.1
    decay_step = 90
    if epoch % decay_step == 0 and epoch:
        return lr * decay_rate
    return lr

callbacks_list = [keras.callbacks.LearningRateScheduler(
                  lr_scheduler, verbose=1)]

history = model.fit(x_train, y_train,
                    epochs=2,
                    batch_size=128,
                    validation_split=0.2,
                    callbacks=callbacks_list)


Epoch 00001: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 1/2
157/157 [==============================] - 76s 481ms/step - loss: 0.1473 - acc: 0.9782 - val_loss: 0.8659 - val_acc: 0.8628

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
Epoch 2/2
157/157 [==============================] - 77s 488ms/step - loss: 0.1310 - acc: 0.9837 - val_loss: 0.9763 - val_acc: 0.8614


Or if you  are not using GPU

In [ ]:
from tensorflow.keras.callbacks import LearningRateScheduler

sd=[]
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = [1,1]

    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        sd.append(step_decay(len(self.losses)))
        print('lr:', step_decay(len(self.losses)))

def step_decay(losses):
    if float(2*np.sqrt(np.array(history.losses[-1])))<0.3:
        lrate=0.01*1/(1+0.1*len(history.losses))
        momentum=0.8
        decay_rate=2e-6
        return lrate
    else:
        lrate=0.1
        return lrate

history=LossHistory()
lrate=LearningRateScheduler(step_decay)

history = model.fit(x_train, y_train,
                    epochs=2,
                    batch_size=128,
                    validation_split=0.2,
                    callbacks=[history,lrate],verbose=2)

Epoch 1/2
lr: 0.1
157/157 - 75s - loss: 7.6010 - acc: 0.5016 - val_loss: 7.7192 - val_acc: 0.4938
Epoch 2/2
lr: 0.1
157/157 - 75s - loss: 7.6010 - acc: 0.5016 - val_loss: 7.7192 - val_acc: 0.4938


# CSVLogger
Callback that streams epoch results to a CSV file.

In [ ]:
from tensorflow.keras.callbacks import CSVLogger

csv_logger = CSVLogger('log.csv', append=True, separator=';')

history = model.fit(x_train, y_train,
                    epochs=2,
                    batch_size=128,
                    validation_split=0.2,
                    callbacks=[csv_logger])

Epoch 1/2
157/157 [==============================] - 76s 486ms/step - loss: 7.6010 - acc: 0.5016 - val_loss: 7.7192 - val_acc: 0.4938
Epoch 2/2
157/157 [==============================] - 76s 485ms/step - loss: 7.6010 - acc: 0.5016 - val_loss: 7.7192 - val_acc: 0.4938
